<a href="https://colab.research.google.com/github/OCCI-Staar/Curve-Maps-Filter-Out-Clouds/blob/main/Curve_Maps_SHP_No_Clouds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note this **WILL** filter out clouds
## Create a standard environment

*   Simply Click [ ] in each script cell below

*   If no errors, continue to 1)

In [3]:
#Just need the required versions from GitHub
!wget https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-Filter-Out-Clouds/main/required-info.txt



--2024-06-20 19:20:06--  https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-Filter-Out-Clouds/main/required-info.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143 [text/plain]
Saving to: ‘required-info.txt.4’

required-info.txt.4 100%[===================>]     143  --.-KB/s    in 0s      

2024-06-20 19:20:06 (2.16 MB/s) - ‘required-info.txt.4’ saved [143/143]



In [2]:
# View the content of required-info.txt
!cat required-info.txt

# Install packages listed in required-info.txt
!pip install -r required-info.txt









<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-f552bab6ce72.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-4589f64a2275.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" medi

In [21]:
# Build environment
!docker build -t maps-docker-image .

ERROR: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [ ]:
# Run environment and we good to start
!docker run -it --rm -v "$(pwd):/app" maps-docker-image

In [3]:
#Needed Libraries
import ee
import time
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from datetime import datetime
from geetools import batch
import ipywidgets as widgets

from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'geetools'

# 1) Authenticate yourself through Google Earth Engine

In [ ]:
#Authenticate the GEE User
ee.Authenticate()
# Verify if GEE server was authenticated
try:
  ee.Initialize()
except ee.EEException as error:
  print(f'ERROR: {error}')

# 2) Supply Satellite, Date Range, Field Name

In [ ]:
satellite = "COPERNICUS/S2_SR_HARMONIZED" # @param ["LANDSAT/LT04/C01/T1_SR", "LANDSAT/LT05/C01/T1_SR", "LANDSAT/LE07/C01/T1_SR", "LANDSAT/LC08/C01/T1_SR", "COPERNICUS/S2_SR_HARMONIZED"]
first_date = '2023-06-09'  #@param {type: "date"}
last_date  = '2023-10-03'  #@param {type: "date"}
field_name = "L1" #@param {type:"string"}

#Preset variables for filtering clouds
CLOUD_FILTER = 100
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

if satellite == "COPERNICUS/S2_SR_HARMONIZED":
      sat = 'S2SR'
if satellite == "LANDSAT/LT04/C01/T1_SR":
      sat = 'LT04'
if satellite == "LANDSAT/LT05/C01/T1_SR":
      sat = 'LT05'
if satellite == "LANDSAT/LE07/C01/T1_SR":
      sat = 'LE07'
if satellite == "LANDSAT/LC08/C01/T1_SR":
      sat = 'LC08'

# 3) Choose Field Boundaries file you want processed.


*   File extension must be **.shp**


In [ ]:
from google.colab import files
shp_file = files.upload()

# 4) Time to process and export images

In [ ]:
#Pulls geometry from the polygon
def get_bbox_coords(pol):
    shp_bb = shp_file[:-5] + '_bb.geojson'
    #Convert polygon CRS to WGS84 DD
    pol = pol.to_crs({'init': 'epsg:4326'})
    #Create a buffer for the boundary and save as geoJSON
    pol.envelope.to_file(filename=shp_bb, driver="GeoJSON")
    #Open GeoJSON
    geom = json.loads(pol.envelope.to_json())['features'][0]['geometry']

    return(geom)

In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter satellite
    s2_sr_col = (ee.ImageCollection(satellite)
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
pol = gpd.read_file(shp_file)

pol = pol.dissolve()

pol = pol.buffer(0.005)

geom = get_bbox_coords(pol)

#Get coordinates from GeoJSON
coords = geom['coordinates']
regions = ee.Geometry.Polygon(coords)

start_date = ee.Date(first_date)
end_date = ee.Date(last_date)

scale = 10
extra = dict(sat=sat)
s2_sr_cld_col = get_s2_sr_cld_col(regions, start_date, end_date)
imagery = s2_sr_cld_col
field_name_sat = f'{field_name}'+"{id}"

print(imagery.size().getInfo())
batch.Export.imagecollection.toDrive(imagery, folder=f'YieldCurve_{sat}_{field_name}',
                                     region=regions, scale=scale, verbose = True,
                                     namePattern = field_name_sat)
time.sleep(300)
print("Export tasks were a success!")